In [1]:
import json
import random
import re
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet

In [ ]:
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('punkt')

In [3]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import f1_score, confusion_matrix, precision_score, recall_score
from torch import nn
from torch import optim

# TF Model Neural Network:-

In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(NeuralNetwork, self).__init__()
        self.layer_1 = nn.Linear(input_dim, 32)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)
        self.layer_2 = nn.Linear(32, 16)
        self.layer_3 = nn.Linear(16, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.layer_1(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.layer_2(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.layer_3(x)

        return x

In [7]:
input_dim = 2
output_dim = 2

class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(NeuralNetwork, self).__init__()
        self.layer_1 = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        x = self.layer_1(x)

        return x

In [8]:
def trainer(X_train, y_train):
    """For training the model"""
    learning_rate = 1e-3
    num_epochs = 30
    batch_size = 32
    loss_values = []

    input_dim = 2
    output_dim = 2

    model = NeuralNetwork(input_dim, output_dim)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-3)

    train_data = TensorDataset(X_train, y_train)
    train_dataloader = DataLoader(dataset=train_data,batch_size=batch_size,shuffle=True)

    for epoch in range(num_epochs):
        for Xt,yt in train_dataloader:
            # In the starting for everyt training data our gradient should be 0
            optimizer.zero_grad()

            pred = model(Xt)  # do the prediction
            loss = loss_fn(pred, yt)  # calculate loss
            loss_values.append(loss.item())   # keep storing the loss
            loss.backward()   # do backpropagation and compute the gradients of all the model parameters
            optimizer.step()   # update the parameters
    return model

In [12]:
def tester(X_test,y_test,model):
    """For testing the model"""
    test_data = TensorDataset(X_test,y_test)

    model.eval()
    correct = 0
    total = 0
    true_labels = []
    predicted_labels = []

    with torch.no_grad():
        for Xt,yt in test_data:
            outputs = model(Xt)

            predicted = torch.argmax(nn.functional.softmax(outputs, dim=-1)).item()

            if (predicted==yt):
                correct+=1
            total+=1
            true_labels.append(yt.item())
            predicted_labels.append(predicted)

    confusions = confusion_matrix(true_labels, predicted_labels)
    print("Confusion Matrix: ", confusions)

    accuracy = correct/total
    print("Accuracy: " + str(accuracy))

    print("F1 Score",f1_score(predicted_labels, true_labels))
    print("Precision",precision_score(predicted_labels, true_labels))
    print("Recall",recall_score(predicted_labels, true_labels))


In [13]:
def probability_distib_finder(review, ai_dict, human_dict,type):
    notations= [['JJ','JJS','JJR'],['NN','NNS','NNP','NNPS'],['RB','RBR','RBS']]

    ai_prob = 0
    human_prob = 0
    words = [tag[0] for tag in pos_tag(word_tokenize(review)) if tag[1] in notations[type]]

    for word in words:
        if word in list(ai_dict.keys()):
            ai_prob += ai_dict[word]
        if word in list(human_dict.keys()):
            human_prob += human_dict[word]

    return [ai_prob, human_prob]

In [14]:
def predictor(ai_iclr_files, ai_neur_files, human_iclr_files, human_neur_files,ai_dict, human_dict):
    """For prediction of a dataset"""
    ai_iclr = list(json.load(open(ai_iclr_files)).values())
    ai_neur = list(json.load(open(ai_neur_files)).values())
    human_iclr = list(json.load(open(human_iclr_files)).values())
    human_neur = list(json.load(open(human_neur_files)).values())

    # Finding probability distributions
    ai_train = [probability_distib_finder(review, ai_dict, human_dict,0) for review in ai_iclr[:608]] + [probability_distib_finder(review, ai_dict, human_dict,0) for review in ai_neur[:576]]
    human_train = [probability_distib_finder(a, ai_dict, human_dict,0) for review in human_iclr[:608] for a in review] + [probability_distib_finder(a, ai_dict, human_dict,0) for review in human_neur[:576] for a in review]

    ai_test_iclr = [probability_distib_finder(review, ai_dict, human_dict,0) for review in ai_iclr[608:]]
    ai_test_neur = [probability_distib_finder(review, ai_dict, human_dict,0) for review in ai_neur[576:]]
    human_test_iclr = [probability_distib_finder(a, ai_dict, human_dict,0) for review in human_iclr[608:] for a in review]
    human_test_neur = [probability_distib_finder(a, ai_dict, human_dict,0) for review in human_neur[576:] for a in review]

    X_train = torch.tensor(ai_train + human_train,dtype=torch.float32)
    y_train = torch.tensor([1 for i in range(len(ai_train))] + [0 for i in range(len(human_train))])

    # Shuffle the train dataset
    perm_indices = torch.randperm(len(X_train))
    X_train = X_train[perm_indices]
    y_train = y_train[perm_indices]

    # Train the model
    model = trainer(X_train, y_train)

    X_test_iclr = torch.tensor(ai_test_iclr + human_test_iclr,dtype=torch.float32)
    X_test_neur = torch.tensor(ai_test_neur + human_test_neur,dtype=torch.float32)
    y_test_iclr = torch.tensor([1 for i in range(len(ai_test_iclr))] + [0 for i in range(len(human_test_iclr))])
    y_test_neur = torch.tensor([1 for i in range(len(ai_test_neur))] + [0 for i in range(len(human_test_neur))])

    # Shuffle the test dataset
    perm_indices = torch.randperm(len(X_test_iclr))
    X_test_iclr = X_test_iclr[perm_indices]
    y_test_iclr = y_test_iclr[perm_indices]

    perm_indices = torch.randperm(len(X_test_neur))
    X_test_neur = X_test_neur[perm_indices]
    y_test_neur = y_test_neur[perm_indices]

    print("++++++++++++++++++ ICLR ++++++++++++++++++")
    tester(X_test_iclr,y_test_iclr,model)
    print("\n++++++++++++++++++ NeurIPS ++++++++++++++++++")
    tester(X_test_neur,y_test_neur,model)


In [15]:
def trainer_tester(ai_iclr_train, ai_neur_train,human_iclr_train, human_neur_train,ai_iclr_test, ai_neur_test,human_iclr_test, human_neur_test,ai_dict, human_dict):
    """For training and testing the model of separate training and testing data"""

    # Training part
    ai_iclr = list(json.load(open(ai_iclr_train)).values())
    ai_neur = list(json.load(open(ai_neur_train)).values())
    human_iclr = list(json.load(open(human_iclr_train)).values())
    human_neur = list(json.load(open(human_neur_train)).values())

    # Finding probability distributions
    ai_train = [probability_distib_finder(review, ai_dict, human_dict,0) for review in ai_iclr[:608]] + [probability_distib_finder(review, ai_dict, human_dict,0) for review in ai_neur[:576]]
    human_train = [probability_distib_finder(a, ai_dict, human_dict,0) for review in human_iclr[:608] for a in review] + [probability_distib_finder(a, ai_dict, human_dict,0) for review in human_neur[:576] for a in review]

    X_train = torch.tensor(ai_train + human_train,dtype=torch.float32)
    y_train = torch.tensor([1 for i in range(len(ai_train))] + [0 for i in range(len(human_train))])

    # Shuffle the train dataset
    perm_indices = torch.randperm(len(X_train))
    X_train = X_train[perm_indices]
    y_train = y_train[perm_indices]

    # Train the model
    model = trainer(X_train, y_train)


    # Testing part
    ai_iclr_test = list(json.load(open(ai_iclr_test)).values())
    ai_neur_test = list(json.load(open(ai_neur_test)).values())
    human_iclr_test = list(json.load(open(human_iclr_test)).values())
    human_neur_test = list(json.load(open(human_neur_test)).values())

    ai_test_iclr = [probability_distib_finder(review, ai_dict, human_dict,0) for review in ai_iclr[608:]]
    ai_test_neur = [probability_distib_finder(review, ai_dict, human_dict,0) for review in ai_neur[576:]]
    human_test_iclr = [probability_distib_finder(a, ai_dict, human_dict,0) for review in human_iclr[608:] for a in review]
    human_test_neur = [probability_distib_finder(a, ai_dict, human_dict,0) for review in human_neur[576:] for a in review]

    X_test_iclr = torch.tensor(ai_test_iclr + human_test_iclr,dtype=torch.float32)
    X_test_neur = torch.tensor(ai_test_neur + human_test_neur,dtype=torch.float32)
    y_test_iclr = torch.tensor([1 for i in range(len(ai_test_iclr))] + [0 for i in range(len(human_test_iclr))])
    y_test_neur = torch.tensor([1 for i in range(len(ai_test_neur))] + [0 for i in range(len(human_test_neur))])

    # Shuffle the test dataset
    perm_indices = torch.randperm(len(X_test_iclr))
    X_test_iclr = X_test_iclr[perm_indices]
    y_test_iclr = y_test_iclr[perm_indices]

    perm_indices = torch.randperm(len(X_test_neur))
    X_test_neur = X_test_neur[perm_indices]
    y_test_neur = y_test_neur[perm_indices]

    print("++++++++++++++++++ ICLR ++++++++++++++++++")
    tester(X_test_iclr,y_test_iclr,model)
    print("\n++++++++++++++++++ NeurIPS ++++++++++++++++++")
    tester(X_test_neur,y_test_neur,model)


## Adjective Model :-
Training the model by considering tokens to be adjective

In [10]:
with open("Dictionaries/ai_adj_dict.json") as f:
    ai_adj_dict = json.load(f)
with open("Dictionaries/human_adj_dict.json") as f:
    human_adj_dict = json.load(f)

In [ ]:
predictor("Dataset/ai_iclr.json","Dataset/ai_neur.json","Dataset/human_iclr.json","Dataset/human_neur.json",ai_adj_dict, human_adj_dict)

## Noun Model:-
Training the model by considering the tokens to be noun

In [33]:
with open("Dictionaries/ai_noun_dict.json") as f:
    ai_noun_dict = json.load(f)
with open("Dictionaries/human_noun_dict.json") as f:
    human_noun_dict = json.load(f)

In [ ]:
predictor("Dataset/ai_iclr.json","Dataset/ai_neur.json","Dataset/human_iclr.json","Dataset/human_neur.json",ai_noun_dict, human_noun_dict)

## Adverb Model:-
Training the model by considering the tokens to be adverb

In [35]:
with open("Dictionaries/ai_adverb_dict.json") as f:
    ai_adverb_dict = json.load(f)
with open("Dictionaries/human_adverb_dict.json") as f:
    human_adverb_dict = json.load(f)

In [ ]:
predictor("Dataset/ai_iclr.json","Dataset/ai_neur.json","Dataset/human_iclr.json","Dataset/human_neur.json",ai_adverb_dict, human_adverb_dict)

## Synonym Attack (or Adjective Attack):-

In [37]:
with open("Dictionaries/ai_adj_dict.json") as f:
    ai_adj_dict = json.load(f)
with open("Dictionaries/human_adj_dict.json") as f:
    human_adj_dict = json.load(f)

In [ ]:
trainer_tester("Dataset/ai_iclr.json","Dataset/ai_neur.json","Dataset/human_iclr.json","Dataset/human_neur.json","Dataset/Attacked Dataset/Adjective Attack/ai_iclr.json","Dataset/Attacked Dataset/Adjective Attack/ai_neur.json","Dataset/Attacked Dataset/Adjective Attack/human_iclr.json","Dataset/Attacked Dataset/Adjective Attack/human_neur.json",ai_adj_dict, human_adj_dict)

## Paraphrasing Attack:-

In [ ]:
with open("Dictionaries/ai_adj_dict.json") as f:
    ai_adj_dict = json.load(f)
with open("Dictionaries/human_adj_dict.json") as f:
    human_adj_dict = json.load(f)

In [ ]:
trainer_tester("Dataset/ai_iclr.json","Dataset/ai_neur.json","Dataset/human_iclr.json","Dataset/human_neur.json","Dataset/Attacked Dataset/Paraphrasing Attack/para_ai_iclr.json","Dataset/Attacked Dataset/Paraphrasing Attack/para_ai_neur.json","Dataset/Attacked Dataset/Paraphrasing Attack/para_human_iclr.json","Dataset/Attacked Dataset/Paraphrasing Attack/para_human_neur.json",ai_adj_dict, human_adj_dict)

## Modified Paraphrasing Attack :-

In [19]:
with open("Dictionaries/ai_adj_dict.json") as f:
    ai_adj_dict = json.load(f)
with open("Dictionaries/human_adj_dict.json") as f:
    human_adj_dict = json.load(f)

In [18]:
ai_iclr = list(json.load(open("Dataset/ai_iclr.json")).values())
ai_neur = list(json.load(open("Dataset/ai_neur.json")).values())
human_iclr = list(json.load(open("Dataset/human_iclr.json")).values())
human_neur = list(json.load(open("Dataset/human_neur.json")).values())

ai_iclr_modified = list(json.load(open("Dataset/Attacked Dataset/Modified Paraphrasing Attack/Modified Review/ai_iclr_modified.json")).values())
ai_neur_modified = list(json.load(open("Dataset/Attacked Dataset/Modified Paraphrasing Attack/Modified Review/ai_neur_modified.json")).values())
human_iclr_modified = list(json.load(open("Dataset/Attacked Dataset/Modified Paraphrasing Attack/Modified Review/human_iclr_modified.json")).values())
human_neur_modified = list(json.load(open("Dataset/Attacked Dataset/Modified Paraphrasing Attack/Modified Review/human_neur_modified.json")).values())

ai_iclr_para_modified = list(json.load(open("Dataset/Attacked Dataset/Modified Paraphrasing Attack/Modified Paraphrased/ai_iclr_para_modified.json")).values())
ai_neur_para_modified = list(json.load(open("Dataset/Attacked Dataset/Modified Paraphrasing Attack/Modified Paraphrased/ai_neur_para_modified.json")).values())
human_iclr_para_modified = list(json.load(open("Dataset/Attacked Dataset/Modified Paraphrasing Attack/Modified Paraphrased/human_iclr_para_modified.json")).values())
human_neur_para_modified = list(json.load(open("Dataset/Attacked Dataset/Modified Paraphrasing Attack/Modified Paraphrased/human_neur_para_modified.json")).values())

In [29]:
# Finding probability distributions
ai_train = [probability_distib_finder(review, ai_adj_dict, human_adj_dict,0) for review in ai_iclr[:608]] + [probability_distib_finder(review, ai_adj_dict, human_adj_dict,0) for review in ai_neur[:576]] +  [probability_distib_finder(review, ai_adj_dict, human_adj_dict,0) for review in ai_iclr_modified[:608]] + [probability_distib_finder(review, ai_adj_dict, human_adj_dict,0) for review in ai_neur_modified[:576]] +  [probability_distib_finder(review, ai_adj_dict, human_adj_dict,0) for review in ai_iclr_para_modified[:608]] + [probability_distib_finder(review, ai_adj_dict, human_adj_dict,0) for review in ai_neur_para_modified[:576]]
human_train = [probability_distib_finder(a, ai_adj_dict, human_adj_dict,1) for review in human_iclr[:608] for a in review] + [probability_distib_finder(a, ai_adj_dict, human_adj_dict,1) for review in human_neur[:576] for a in review] +  [probability_distib_finder(a, ai_adj_dict, human_adj_dict,1) for review in human_iclr_modified[:608] for a in review] + [probability_distib_finder(a, ai_adj_dict, human_adj_dict,1) for review in human_neur_modified[:576] for a in review] +  [probability_distib_finder(a, ai_adj_dict, human_adj_dict,1) for review in human_iclr_para_modified[:608] for a in review] + [probability_distib_finder(a, ai_adj_dict, human_adj_dict,1) for review in human_neur_para_modified[:576] for a in review]

X_train = torch.tensor(ai_train + human_train,dtype=torch.float32)
y_train = torch.tensor([1 for i in range(len(ai_train))] + [0 for i in range(len(human_train))])

# Shuffle the train dataset
perm_indices = torch.randperm(len(X_train))
X_train = X_train[perm_indices]
y_train = y_train[perm_indices]

# Train the model
model = trainer(X_train, y_train)

### Testing the Modified Paraphrased Dataset:-

In [ ]:
# Testing only modified paraphrased part
ai_test_iclr =    [probability_distib_finder(review, ai_adj_dict, human_adj_dict,0) for review in ai_iclr_para_modified[608:]]
human_test_iclr =    [probability_distib_finder(a, ai_adj_dict, human_adj_dict,1) for review in human_iclr_para_modified[608:] for a in review]

ai_test_neur = [probability_distib_finder(review, ai_adj_dict, human_adj_dict,0) for review in ai_neur[576:]] +  [probability_distib_finder(review, ai_adj_dict, human_adj_dict,0) for review in ai_neur_modified[576:]] +  [probability_distib_finder(review, ai_adj_dict, human_adj_dict,0) for review in ai_neur_para_modified[576:]]
human_test_neur = [probability_distib_finder(a, ai_adj_dict, human_adj_dict,1) for review in human_neur[576:] for a in review] +  [probability_distib_finder(a, ai_adj_dict, human_adj_dict,1) for review in human_neur_modified[576:] for a in review]  +  [probability_distib_finder(a, ai_adj_dict, human_adj_dict,1) for review in human_neur_para_modified[576:] for a in review]

X_test_iclr = torch.tensor(ai_test_iclr + human_test_iclr,dtype=torch.float32)
X_test_neur = torch.tensor(ai_test_neur + human_test_neur,dtype=torch.float32)
y_test_iclr = torch.tensor([1 for i in range(len(ai_test_iclr))] + [0 for i in range(len(human_test_iclr))])
y_test_neur = torch.tensor([1 for i in range(len(ai_test_neur))] + [0 for i in range(len(human_test_neur))])

# Shuffle the test dataset
perm_indices = torch.randperm(len(X_test_iclr))
X_test_iclr = X_test_iclr[perm_indices]
y_test_iclr = y_test_iclr[perm_indices]

perm_indices = torch.randperm(len(X_test_neur))
X_test_neur = X_test_neur[perm_indices]
y_test_neur = y_test_neur[perm_indices]

print("++++++++++++++++++ ICLR ++++++++++++++++++")
tester(X_test_iclr,y_test_iclr,model)
print("\n++++++++++++++++++ NeurIPS ++++++++++++++++++")
tester(X_test_neur,y_test_neur,model)

### Testing the Modified Actual Review Dataset :-

In [ ]:
# Testing only modified actual review
ai_test_iclr =   [probability_distib_finder(review, ai_adj_dict, human_adj_dict,0) for review in ai_iclr_modified[608:]]
human_test_iclr =  [probability_distib_finder(a, ai_adj_dict, human_adj_dict,1) for review in human_iclr_modified[608:] for a in review]

ai_test_neur = [probability_distib_finder(review, ai_adj_dict, human_adj_dict,0) for review in ai_neur[576:]] +  [probability_distib_finder(review, ai_adj_dict, human_adj_dict,0) for review in ai_neur_modified[576:]] +  [probability_distib_finder(review, ai_adj_dict, human_adj_dict,0) for review in ai_neur_para_modified[576:]]
human_test_neur = [probability_distib_finder(a, ai_adj_dict, human_adj_dict,1) for review in human_neur[576:] for a in review] +  [probability_distib_finder(a, ai_adj_dict, human_adj_dict,1) for review in human_neur_modified[576:] for a in review]  +  [probability_distib_finder(a, ai_adj_dict, human_adj_dict,1) for review in human_neur_para_modified[576:] for a in review]

X_test_iclr = torch.tensor(ai_test_iclr + human_test_iclr,dtype=torch.float32)
X_test_neur = torch.tensor(ai_test_neur + human_test_neur,dtype=torch.float32)
y_test_iclr = torch.tensor([1 for i in range(len(ai_test_iclr))] + [0 for i in range(len(human_test_iclr))])
y_test_neur = torch.tensor([1 for i in range(len(ai_test_neur))] + [0 for i in range(len(human_test_neur))])

# Shuffle the test dataset
perm_indices = torch.randperm(len(X_test_iclr))
X_test_iclr = X_test_iclr[perm_indices]
y_test_iclr = y_test_iclr[perm_indices]

perm_indices = torch.randperm(len(X_test_neur))
X_test_neur = X_test_neur[perm_indices]
y_test_neur = y_test_neur[perm_indices]

print("++++++++++++++++++ ICLR ++++++++++++++++++")
tester(X_test_iclr,y_test_iclr,model)
print("\n++++++++++++++++++ NeurIPS ++++++++++++++++++")
tester(X_test_neur,y_test_neur,model)

### Testing by combining all of them :-

In [ ]:
# By combining them all
ai_test_iclr = [probability_distib_finder(review, ai_adj_dict, human_adj_dict,0) for review in ai_iclr[608:]] +  [probability_distib_finder(review, ai_adj_dict, human_adj_dict,0) for review in ai_iclr_modified[608:]] +  [probability_distib_finder(review, ai_adj_dict, human_adj_dict,0) for review in ai_iclr_para_modified[608:]]
human_test_iclr = [probability_distib_finder(a, ai_adj_dict, human_adj_dict,1) for review in human_iclr[608:] for a in review] +  [probability_distib_finder(a, ai_adj_dict, human_adj_dict,1) for review in human_iclr_modified[608:] for a in review]  +  [probability_distib_finder(a, ai_adj_dict, human_adj_dict,1) for review in human_iclr_para_modified[608:] for a in review]

ai_test_neur = [probability_distib_finder(review, ai_adj_dict, human_adj_dict,0) for review in ai_neur[576:]] +  [probability_distib_finder(review, ai_adj_dict, human_adj_dict,0) for review in ai_neur_modified[576:]] +  [probability_distib_finder(review, ai_adj_dict, human_adj_dict,0) for review in ai_neur_para_modified[576:]]
human_test_neur = [probability_distib_finder(a, ai_adj_dict, human_adj_dict,1) for review in human_neur[576:] for a in review] +  [probability_distib_finder(a, ai_adj_dict, human_adj_dict,1) for review in human_neur_modified[576:] for a in review]  +  [probability_distib_finder(a, ai_adj_dict, human_adj_dict,1) for review in human_neur_para_modified[576:] for a in review]

X_test_iclr = torch.tensor(ai_test_iclr + human_test_iclr,dtype=torch.float32)
X_test_neur = torch.tensor(ai_test_neur + human_test_neur,dtype=torch.float32)
y_test_iclr = torch.tensor([1 for i in range(len(ai_test_iclr))] + [0 for i in range(len(human_test_iclr))])
y_test_neur = torch.tensor([1 for i in range(len(ai_test_neur))] + [0 for i in range(len(human_test_neur))])

# Shuffle the test dataset
perm_indices = torch.randperm(len(X_test_iclr))
X_test_iclr = X_test_iclr[perm_indices]
y_test_iclr = y_test_iclr[perm_indices]

perm_indices = torch.randperm(len(X_test_neur))
X_test_neur = X_test_neur[perm_indices]
y_test_neur = y_test_neur[perm_indices]

print("++++++++++++++++++ ICLR ++++++++++++++++++")
tester(X_test_iclr,y_test_iclr,model)
print("\n++++++++++++++++++ NeurIPS ++++++++++++++++++")
tester(X_test_neur,y_test_neur,model)